In [194]:
!pip install apyori
import pandas as pd 
import numpy as np
from apyori import apriori

In [195]:
RetailBankEFG = pd.read_csv('RetailBankEFG.csv')
InvestmentBankCDE = pd.read_csv('InvestmentBankCDE.csv')
InsuranceCompanyABC = pd.read_csv('InsuranceCompanyABC.csv')
ANSWERS = pd.read_csv('ANSWERS.csv')

In [196]:
RetailBankEFG.head()

,ID,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos
0,71504.0,T,T,T,T,T,F,F,T,F
1,59751.0,T,T,T,T,T,F,F,F,T
2,11877.0,F,F,F,F,F,F,F,F,F
3,33701.0,F,F,F,F,F,F,F,F,F
4,62029.0,F,T,T,T,T,T,T,T,T


In [197]:
InvestmentBankCDE.head()

,ID,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto
0,71504.0,F,F,F,F,F,F,F
1,59751.0,F,F,F,F,F,F,F
2,11877.0,F,F,F,F,F,F,F
3,33701.0,T,T,T,T,T,T,T
4,62029.0,F,F,T,F,T,T,F


In [198]:
InsuranceCompanyABC.head()

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial
0,71504.0,42.993,4428.512,NE,1.0,T,T,T,T
1,59751.0,26.373,4489.930,CO,0.0,T,T,T,T
2,11877.0,46.977,4567.741,CO,0.0,F,F,F,F
3,33701.0,24.311,4592.132,NE,0.0,F,F,F,F
4,62029.0,28.896,4723.197,SE,1.0,F,F,F,F


In [199]:
insurances = InsuranceCompanyABC[['ID', 'seguro auto', 'seguro vida Emp', 'seguro vida PF', 'Seguro Residencial']]
insurances.head()

,ID,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial
0,71504.0,T,T,T,T
1,59751.0,T,T,T,T
2,11877.0,F,F,F,F
3,33701.0,F,F,F,F
4,62029.0,F,F,F,F


In [200]:
df = pd.merge(RetailBankEFG, InvestmentBankCDE, how='outer', on='ID')
df = pd.merge(df, insurances, how='outer', on='ID')
df = df.drop(columns=['ID'])
df.head()

,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial
0,T,T,T,T,T,F,F,T,F,F,F,F,F,F,F,F,T,T,T,T
1,T,T,T,T,T,F,F,F,T,F,F,F,F,F,F,F,T,T,T,T
2,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F
3,F,F,F,F,F,F,F,F,F,T,T,T,T,T,T,T,F,F,F,F
4,F,T,T,T,T,T,T,T,T,F,F,T,F,T,T,F,F,F,F,F


In [201]:
columns = df.columns
columns

Index(['Financiamento Casa', 'Financiamento Carro', 'Emprestimo _pessoal',
       'Emprestimo _consignado', 'Emprestimo _limite_especial',
       'Emprestimo _educacao', 'Emprestimo _viagem', 'Investimento CDB',
       'Investimento Fundos', 'Investimento Fundos_cambiais',
       'Investimento Fundos_commodities', 'Investimento LCI',
       'Investimento LCA', 'Investimento Poupanca',
       'Investimento Fundos Multimercado', 'Investimento Tesouro Direto',
       'seguro auto', 'seguro vida Emp', 'seguro vida PF',
       'Seguro Residencial'],
      dtype='object')

In [202]:
transactions = []
for i in range(len(df)):
  transactions.append([columns[j] for j in range(df.shape[1]) if df.values[i, j] == 'T'])

In [203]:
rules = apriori(transactions, min_support=0.1, min_confidence=0.8, min_lift=5)
rules = list(rules)
rules

[RelationRecord(items=frozenset({'Emprestimo _pessoal', 'Emprestimo _consignado', 'Emprestimo _limite_especial'}), support=0.11951930080116534, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Emprestimo _pessoal', 'Emprestimo _consignado'}), items_add=frozenset({'Emprestimo _limite_especial'}), confidence=0.972732661529342, lift=6.029625030608517), OrderedStatistic(items_base=frozenset({'Emprestimo _pessoal', 'Emprestimo _limite_especial'}), items_add=frozenset({'Emprestimo _consignado'}), confidence=0.9535153980244044, lift=5.931928597587255)]),
 RelationRecord(items=frozenset({'Investimento Fundos', 'Emprestimo _educacao', 'Emprestimo _viagem'}), support=0.11828113619810633, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Investimento Fundos', 'Emprestimo _viagem'}), items_add=frozenset({'Emprestimo _educacao'}), confidence=0.8011840157868771, lift=5.009224288139263)]),
 RelationRecord(items=frozenset({'Investimento Fundos', 'Investimento CDB', 'Emprestimo _ed

In [204]:
A = []
B = []
support = []
confidence = []
lift = []
for r in rules:
  sup = r[1]
  statistics = r[2]
  for s in statistics:
    a = list(s[0])
    b = list(s[1])
    c = s[2]
    l = s[3]
    A.append(a)
    B.append(b)
    support.append(sup)
    confidence.append(c)
    lift.append(l)


In [205]:
rules_df = pd.DataFrame({'A': A, 'B': B, 'support': support, 'confidence': confidence, 'lift': lift})
rules_df

,A,B,support,confidence,lift
0,"[Emprestimo _pessoal, Emprestimo _consignado]",[Emprestimo _limite_especial],0.119519,0.972733,6.029625
1,"[Emprestimo _pessoal, Emprestimo _limite_espec...",[Emprestimo _consignado],0.119519,0.953515,5.931929
2,"[Investimento Fundos, Emprestimo _viagem]",[Emprestimo _educacao],0.118281,0.801184,5.009224
3,"[Emprestimo _educacao, Investimento CDB]","[Investimento Fundos, Emprestimo _viagem]",0.115222,0.863066,5.846027
4,"[Investimento Fundos, Emprestimo _educacao]","[Emprestimo _viagem, Investimento CDB]",0.115222,0.896825,5.296091
5,"[Investimento Fundos, Investimento CDB, Empres...",[Emprestimo _educacao],0.115222,0.887269,5.547449
6,"[Investimento Fundos_cambiais, Investimento Po...","[Investimento LCA, Investimento LCI]",0.121413,0.829766,5.108829
7,"[Investimento Fundos_cambiais, Investimento Po...","[Investimento Fundos_commodities, Investimento...",0.121413,0.837688,5.305103
8,"[Investimento Poupanca, Investimento Fundos_co...","[Investimento Fundos_cambiais, Investimento LCA]",0.121413,0.826065,5.083764


In [206]:
A

[['Emprestimo _pessoal', 'Emprestimo _consignado'],
 ['Emprestimo _pessoal', 'Emprestimo _limite_especial'],
 ['Investimento Fundos', 'Emprestimo _viagem'],
 ['Emprestimo _educacao', 'Investimento CDB'],
 ['Investimento Fundos', 'Emprestimo _educacao'],
 ['Investimento Fundos', 'Investimento CDB', 'Emprestimo _viagem'],
 ['Investimento Fundos_cambiais',
  'Investimento Poupanca',
  'Investimento Fundos_commodities'],
 ['Investimento Fundos_cambiais', 'Investimento Poupanca', 'Investimento LCI'],
 ['Investimento Poupanca',
  'Investimento Fundos_commodities',
  'Investimento LCI']]

In [207]:
response = ANSWERS.copy()

rule_1 = (response['Emprestimo _pessoal'] == 'T') & (response['Emprestimo _consignado'] == 'T') 
rule_2 = (response['Emprestimo _pessoal'] == 'T') & (response['Emprestimo _limite_especial'] == 'T')
rule_3 = (response['Investimento Fundos'] == 'T') & (response['Emprestimo _viagem'] == 'T')
rule_4 = (response['Emprestimo _educacao'] == 'T') & (response['Investimento CDB'] == 'T')
rule_5 = (response['Investimento Fundos'] == 'T') & (response['Emprestimo _educacao'] == 'T')
rule_6 = (response['Investimento Fundos'] == 'T') & (response['Investimento CDB'] == 'T') & (response['Emprestimo _viagem'] == 'T')
rule_7 = (response['Investimento Fundos_cambiais'] == 'T') & (response['Investimento Poupanca'] == 'T') & (response['Investimento Fundos_commodities'] == 'T')
rule_8 = (response['Investimento Fundos_cambiais'] == 'T') & (response['Investimento Poupanca'] == 'T') & (response['Investimento LCI'] == 'T')
rule_9 = (response['Investimento Fundos_commodities'] == 'T') & (response['Investimento Poupanca'] == 'T') & (response['Investimento LCI'] == 'T')

for x in range(1, 2): 
  i = str(x)
  response.loc[rule_1 & (response['RECOMMENDATION_' + i].isnull()), 'RECOMMENDATION_' + i] = 'Emprestimo _limite_especial'
  response.loc[rule_1 & (response['CONFIDENCE_' + i].isnull()), 'CONFIDENCE_' + i] = 0.972733

  response.loc[rule_2 & (response['RECOMMENDATION_' + i].isnull()), 'RECOMMENDATION_' + i] = 'Emprestimo _consignado'
  response.loc[rule_2 & (response['CONFIDENCE_' + i].isnull()), 'CONFIDENCE_' + i] = 0.953515

  response.loc[rule_3 & (response['RECOMMENDATION_' + i].isnull()), 'RECOMMENDATION_' + i] = 'Emprestimo _educacao'
  response.loc[rule_3 & (response['CONFIDENCE_' + i].isnull()), 'CONFIDENCE_' + i] = 0.801184

  response.loc[rule_4 & (response['RECOMMENDATION_' + i].isnull()), 'RECOMMENDATION_' + i] = 'Investimento Fundos'
  response.loc[rule_4 & (response['CONFIDENCE_' + i].isnull()), 'CONFIDENCE_' + i] = 0.863066

  response.loc[rule_5 & (response['RECOMMENDATION_' + i].isnull()), 'RECOMMENDATION_' + i] = 'Emprestimo _viagem'
  response.loc[rule_5 & (response['CONFIDENCE_' + i].isnull()), 'CONFIDENCE_' + i] = 0.896825

  response.loc[rule_6 & (response['RECOMMENDATION_' + i].isnull()), 'RECOMMENDATION_' + i] = 'Emprestimo _educacao'
  response.loc[rule_6 & (response['CONFIDENCE_' + i].isnull()), 'CONFIDENCE_' + i] = 0.887269

  response.loc[rule_7 & (response['RECOMMENDATION_' + i].isnull()), 'RECOMMENDATION_' + i] = 'Investimento LCA'
  response.loc[rule_7 & (response['CONFIDENCE_' + i].isnull()), 'CONFIDENCE_' + i] = 0.829766

  response.loc[rule_8 & (response['RECOMMENDATION_' + i].isnull()), 'RECOMMENDATION_' + i] = 'Investimento Fundos_commodities'
  response.loc[rule_8 & (response['CONFIDENCE_' + i].isnull()), 'CONFIDENCE_' + i] = 0.837688

  response.loc[rule_9 & (response['RECOMMENDATION_' + i].isnull()), 'RECOMMENDATION_' + i] = 'Investimento Fundos_cambiais'
  response.loc[rule_9 & (response['CONFIDENCE_' + i].isnull()), 'CONFIDENCE_' + i] = 0.826065

In [208]:
for x in range(2, 3): 
  i = str(x)
  j = str(x - 1)
  response.loc[rule_1 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Emprestimo _limite_especial'), 'RECOMMENDATION_' + i] = 'Emprestimo _limite_especial'
  response.loc[rule_1 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.972733), 'CONFIDENCE_' + i] = 0.972733

  response.loc[rule_2 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Emprestimo _consignado'), 'RECOMMENDATION_' + i] = 'Emprestimo _consignado'
  response.loc[rule_2 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.953515), 'CONFIDENCE_' + i] = 0.953515

  response.loc[rule_3 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Emprestimo _educacao'), 'RECOMMENDATION_' + i] = 'Emprestimo _educacao'
  response.loc[rule_3 & (response['CONFIDENCE_' + i].isnull())  & (response['CONFIDENCE_' + j] != 0.801184), 'CONFIDENCE_' + i] = 0.801184

  response.loc[rule_4 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Investimento Fundos'), 'RECOMMENDATION_' + i] = 'Investimento Fundos'
  response.loc[rule_4 & (response['CONFIDENCE_' + i].isnull())  & (response['CONFIDENCE_' + j] != 0.863066), 'CONFIDENCE_' + i] = 0.863066

  response.loc[rule_5 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Emprestimo _viagem'), 'RECOMMENDATION_' + i] = 'Emprestimo _viagem'
  response.loc[rule_5 & (response['CONFIDENCE_' + i].isnull())  & (response['CONFIDENCE_' + j] != 0.896825), 'CONFIDENCE_' + i] = 0.896825

  response.loc[rule_6 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Emprestimo _educacao'), 'RECOMMENDATION_' + i] = 'Emprestimo _educacao'
  response.loc[rule_6 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.887269), 'CONFIDENCE_' + i] = 0.887269

  response.loc[rule_7 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Investimento LCA'), 'RECOMMENDATION_' + i] = 'Investimento LCA'
  response.loc[rule_7 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.829766), 'CONFIDENCE_' + i] = 0.829766

  response.loc[rule_8 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Investimento Fundos_commodities'), 'RECOMMENDATION_' + i] = 'Investimento Fundos_commodities'
  response.loc[rule_8 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.837688), 'CONFIDENCE_' + i] = 0.837688

  response.loc[rule_9 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Investimento Fundos_cambiais'), 'RECOMMENDATION_' + i] = 'Investimento Fundos_cambiais'
  response.loc[rule_9 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.826065), 'CONFIDENCE_' + i] = 0.826065

In [209]:
for x in range(3, 4): 
  i = str(x)
  j = str(x - 1)
  z = str(x - 2)
  response.loc[rule_1 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Emprestimo _limite_especial') & (response['RECOMMENDATION_' + z] != 'Emprestimo _limite_especial'), 'RECOMMENDATION_' + i] = 'Emprestimo _limite_especial'
  response.loc[rule_1 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.972733) & (response['CONFIDENCE_' + z] != 0.972733), 'CONFIDENCE_' + i] = 0.972733

  response.loc[rule_2 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Emprestimo _consignado') & (response['RECOMMENDATION_' + z] != 'Emprestimo _consignado'), 'RECOMMENDATION_' + i] = 'Emprestimo _consignado'
  response.loc[rule_2 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.953515) & (response['CONFIDENCE_' + z] != 0.953515), 'CONFIDENCE_' + i] = 0.953515

  response.loc[rule_3 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Emprestimo _educacao') & (response['RECOMMENDATION_' + z] != 'Emprestimo _educacao'), 'RECOMMENDATION_' + i] = 'Emprestimo _educacao'
  response.loc[rule_3 & (response['CONFIDENCE_' + i].isnull())  & (response['CONFIDENCE_' + j] != 0.801184)  & (response['CONFIDENCE_' + z] != 0.801184), 'CONFIDENCE_' + i] = 0.801184

  response.loc[rule_4 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Investimento Fundos') & (response['RECOMMENDATION_' + z] != 'Investimento Fundos'), 'RECOMMENDATION_' + i] = 'Investimento Fundos'
  response.loc[rule_4 & (response['CONFIDENCE_' + i].isnull())  & (response['CONFIDENCE_' + j] != 0.863066) & (response['CONFIDENCE_' + z] != 0.863066), 'CONFIDENCE_' + i] = 0.863066

  response.loc[rule_5 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Emprestimo _viagem') & (response['RECOMMENDATION_' + z] != 'Emprestimo _viagem'), 'RECOMMENDATION_' + i] = 'Emprestimo _viagem'
  response.loc[rule_5 & (response['CONFIDENCE_' + i].isnull())  & (response['CONFIDENCE_' + j] != 0.896825) & (response['CONFIDENCE_' + z] != 0.896825), 'CONFIDENCE_' + i] = 0.896825

  response.loc[rule_6 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Emprestimo _educacao')  & (response['RECOMMENDATION_' + z] != 'Emprestimo _educacao'), 'RECOMMENDATION_' + i] = 'Emprestimo _educacao'
  response.loc[rule_6 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.887269) & (response['CONFIDENCE_' + z] != 0.887269), 'CONFIDENCE_' + i] = 0.887269

  response.loc[rule_7 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Investimento LCA') & (response['RECOMMENDATION_' + z] != 'Investimento LCA'), 'RECOMMENDATION_' + i] = 'Investimento LCA'
  response.loc[rule_7 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.829766) & (response['CONFIDENCE_' + z] != 0.829766), 'CONFIDENCE_' + i] = 0.829766

  response.loc[rule_8 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Investimento Fundos_commodities') & (response['RECOMMENDATION_' + z] != 'Investimento Fundos_commodities'), 'RECOMMENDATION_' + i] = 'Investimento Fundos_commodities'
  response.loc[rule_8 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.837688) & (response['CONFIDENCE_' + z] != 0.837688), 'CONFIDENCE_' + i] = 0.837688

  response.loc[rule_9 & (response['RECOMMENDATION_' + i].isnull()) & (response['RECOMMENDATION_' + j] != 'Investimento Fundos_cambiais') & (response['RECOMMENDATION_' + z] != 'Investimento Fundos_cambiais'), 'RECOMMENDATION_' + i] = 'Investimento Fundos_cambiais'
  response.loc[rule_9 & (response['CONFIDENCE_' + i].isnull()) & (response['CONFIDENCE_' + j] != 0.826065) & (response['CONFIDENCE_' + z] != 0.826065), 'CONFIDENCE_' + i] = 0.826065

In [222]:
response[response['RECOMMENDATION_3'] == 'Investimento LCA']

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3
120,2139.0,52.772,5169.263,N,1.0,F,T,F,F,T,T,T,T,T,T,T,F,F,F,F,F,T,F,T,T,Investimento Fundos,0.863066,Emprestimo _viagem,0.896825,Investimento LCA,0.829766
277,3522.0,27.064,8581.761,NE,0.0,T,T,T,T,T,T,T,T,T,T,T,F,T,T,T,T,F,F,T,F,Emprestimo _limite_especial,0.972733,Emprestimo _consignado,0.953515,Investimento LCA,0.829766
899,8320.0,43.091,6714.705,SE,0.0,F,F,F,T,T,T,T,T,T,T,T,F,F,F,F,F,T,F,T,T,Investimento Fundos,0.863066,Emprestimo _viagem,0.896825,Investimento LCA,0.829766
1024,9322.0,31.020,6237.877,NE,0.0,T,T,T,T,T,T,F,F,T,F,F,T,T,T,T,T,F,F,F,F,Emprestimo _limite_especial,0.972733,Emprestimo _consignado,0.953515,Investimento LCA,0.829766
1037,9441.0,34.063,7118.671,SE,1.0,T,F,F,F,T,T,T,T,T,F,F,F,F,F,F,T,T,F,T,T,Investimento Fundos,0.863066,Emprestimo _viagem,0.896825,Investimento LCA,0.829766
1115,9954.0,35.834,8457.603,S,1.0,F,T,F,F,T,T,T,T,T,T,T,F,F,F,F,F,T,F,T,T,Investimento Fundos,0.863066,Emprestimo _viagem,0.896825,Investimento LCA,0.829766
1192,10658.0,34.159,8071.961,SE,1.0,F,T,F,F,T,T,T,T,T,F,F,F,F,F,F,F,T,F,T,T,Investimento Fundos,0.863066,Emprestimo _viagem,0.896825,Investimento LCA,0.829766
1243,11036.0,30.587,7303.641,SE,1.0,T,T,T,T,T,T,T,T,T,T,T,F,F,T,T,T,F,T,F,F,Emprestimo _limite_especial,0.972733,Emprestimo _consignado,0.953515,Investimento LCA,0.829766
1476,13026.0,42.055,8159.200,N,1.0,F,F,F,T,T,T,T,T,T,T,T,T,T,T,T,T,F,F,F,T,Emprestimo _limite_especial,0.972733,Emprestimo _consignado,0.953515,Investimento LCA,0.829766
1478,13026.0,21.479,6059.284,NE,1.0,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,T,F,F,F,T,Emprestimo _limite_especial,0.972733,Emprestimo _consignado,0.953515,Investimento LCA,0.829766


In [218]:
response.to_csv('result.csv', index=False)